In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from datetime import datetime
from app.data_filler import fill_sheet_column
from app.date_formatter import DateFormatter

from_date = datetime(2025, 3, 1)
to_date = datetime(2025, 3, 11)
SPREADSHEET_NAME = "rnp-experiment"
WORKSHEET_NAME = "task"

report_dates = DateFormatter.generate_date_range(from_date, to_date)
report_dates
for report_date in report_dates:
    fill_sheet_column(SPREADSHEET_NAME, WORKSHEET_NAME, report_date)

[datetime.datetime(2025, 3, 1, 0, 0),
 datetime.datetime(2025, 3, 2, 0, 0),
 datetime.datetime(2025, 3, 3, 0, 0),
 datetime.datetime(2025, 3, 4, 0, 0),
 datetime.datetime(2025, 3, 5, 0, 0),
 datetime.datetime(2025, 3, 6, 0, 0),
 datetime.datetime(2025, 3, 7, 0, 0),
 datetime.datetime(2025, 3, 8, 0, 0),
 datetime.datetime(2025, 3, 9, 0, 0),
 datetime.datetime(2025, 3, 10, 0, 0),
 datetime.datetime(2025, 3, 11, 0, 0)]